In [10]:
import os, glob, json, time, math, random
from dataclasses import dataclass
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

### Pointing towards the dataset

In [11]:
DATA_DIR = "7890488"
CITY_INFO_PATH = os.path.join(DATA_DIR, "city_info.csv")

assert os.path.exists(DATA_DIR), f"Missing folder: {DATA_DIR}"
assert os.path.exists(CITY_INFO_PATH), f"Missing file: {CITY_INFO_PATH}"

city_info = pd.read_csv(CITY_INFO_PATH)
city_info.head(), city_info.shape

(   Unnamed: 0      Name           ID      Lat       Lon  \
 0           1    Lander  USW00024021  42.8153 -108.7261   
 1           2    Lander  USW00024021  42.8153 -108.7261   
 2           3  Cheyenne  USW00024018  41.1519 -104.8061   
 3           4  Cheyenne  USW00024018  41.1519 -104.8061   
 4           5    Wausau  USW00014897  44.9258  -89.6256   
 
                   Stn.Name  Stn.stDate  Stn.edDate  
 0               LANDER WBO  1892-01-01  1946-05-28  
 1        LANDER HUNT FIELD  1946-05-29  2023-12-31  
 2             CHEYENNE WBO  1871-01-01  1935-08-31  
 3  CHEYENNE MUNICIPAL ARPT  1935-09-01  2023-12-31  
 4     Wausau Record Herald  1896-01-01  1941-12-31  ,
 (461, 8))

### Identify individual city files

In [4]:
all_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))
city_files = [p for p in all_files if os.path.basename(p).lower() != "city_info.csv"]

print("Total CSV files:", len(all_files))
print("City files:", len(city_files))
print("Example:", os.path.basename(city_files[0]) if city_files else None)

Total CSV files: 211
City files: 210
Example: USW00003822.csv


### Load one city at a time 

In [5]:
sample_path = city_files[0]
df0 = pd.read_csv(sample_path)
df0.head(), df0.columns, df0.shape

(   Unnamed: 0        Date  tmax  tmin  prcp
 0           1  1871-01-01   NaN   NaN   0.0
 1           2  1871-01-02   NaN   NaN   0.0
 2           3  1871-01-03   NaN   NaN   0.0
 3           4  1871-01-04   NaN   NaN   0.0
 4           5  1871-01-05   NaN   NaN   0.0,
 Index(['Unnamed: 0', 'Date', 'tmax', 'tmin', 'prcp'], dtype='object'),
 (55882, 5))

### Handling common naming variations

In [7]:
def standardize_city_df(df: pd.DataFrame, city_id: str) -> pd.DataFrame:
    cols = {c.lower(): c for c in df.columns}

    date_col = cols.get("date")
    tmax_col = cols.get("tmax") 
    tmin_col = cols.get("tmin")
    prcp_col = cols.get("prcp")

    out = df[[date_col, tmax_col, tmin_col, prcp_col]].copy()
    out.columns = ["date", "tmax", "tmin", "prcp"]
    out["date"] = pd.to_datetime(out["date"])
    out["city_id"] = city_id

    for c in ["tmax", "tmin", "prcp"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")

    out = out.sort_values("date").reset_index(drop=True)
    return out

### Loading `n` cities

In [8]:
def city_id_from_path(p: str) -> str:
    return os.path.splitext(os.path.basename(p))[0]

N_CITIES = 25
selected_paths = city_files[:N_CITIES]

frames = []
for p in tqdm(selected_paths):
    cid = city_id_from_path(p)
    df = pd.read_csv(p)
    frames.append(standardize_city_df(df, cid))

data = pd.concat(frames, ignore_index=True)
data.head(), data.shape

  0%|          | 0/25 [00:00<?, ?it/s]

(        date  tmax  tmin  prcp      city_id
 0 1871-01-01   NaN   NaN   0.0  USW00003822
 1 1871-01-02   NaN   NaN   0.0  USW00003822
 2 1871-01-03   NaN   NaN   0.0  USW00003822
 3 1871-01-04   NaN   NaN   0.0  USW00003822
 4 1871-01-05   NaN   NaN   0.0  USW00003822,
 (1273229, 5))

In [9]:
lower_cols = {c.lower(): c for c in city_info.columns}
print("city_info columns:", city_info.columns.tolist())

CITY_ID_COL = lower_cols.get("id")
LAT_COL = lower_cols.get("lat")
LON_COL = lower_cols.get("lon")

assert CITY_ID_COL and LAT_COL and LON_COL, "Fix CITY_ID_COL/LAT_COL/LON_COL based on city_info columns."

meta = city_info[[CITY_ID_COL, LAT_COL, LON_COL]].copy()
meta.columns = ["city_id", "lat", "lon"]

data = data.merge(meta, on="city_id", how="left")
data[["lat","lon"]].isna().mean()

city_info columns: ['Unnamed: 0', 'Name', 'ID', 'Lat', 'Lon', 'Stn.Name', 'Stn.stDate', 'Stn.edDate']


lat    0.0
lon    0.0
dtype: float64

### Feature engineering

In [ ]:
def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d["year"] = d["date"].dt.year
    d["month"] = d["date"].dt.month
    d["dayofyear"] = d["date"].dt.dayofyear
    d["sin_doy"] = np.sin(2*np.pi*d["dayofyear"]/365.25)
    d["cos_doy"] = np.cos(2*np.pi*d["dayofyear"]/365.25)
    return d

def add_lags(df: pd.DataFrame, lags=(1, 7, 30)) -> pd.DataFrame:
    d = df.copy()
    d = d.sort_values(["city_id","date"])
    for lag in lags:
        for col in ["tmax","tmin","prcp"]:
            d[f"{col}_lag{lag}"] = d.groupby("city_id")[col].shift(lag)
    return d

data_fe = add_time_features(data)
data_fe = add_lags(data_fe, lags=(1, 7, 30))

# this targets the next day
for col in ["tmax","tmin","prcp"]:
    data_fe[f"{col}_target"] = data_fe.groupby("city_id")[col].shift(-1)

feature_cols = ["lat","lon","sin_doy","cos_doy"] + \
              [f"{c}_lag{l}" for l in (1,7,30) for c in ("tmax","tmin","prcp")]
target_cols = ["tmax_target","tmin_target","prcp_target"]

model_df = data_fe.dropna(subset=feature_cols + target_cols).reset_index(drop=True)
model_df.head(), model_df.shape

### Tine based split

In [ ]:
dates = model_df["date"]
train_end = pd.Timestamp("2016-12-31")
val_end   = pd.Timestamp("2019-12-31")

train_df = model_df[dates <= train_end].copy()
val_df   = model_df[(dates > train_end) & (dates <= val_end)].copy()
test_df  = model_df[dates > val_end].copy()

len(train_df), len(val_df), len(test_df), (train_df["date"].min(), test_df["date"].max())

### Scaling features

In [ ]:
X_train = train_df[feature_cols].values.astype(np.float32)
y_train = train_df[target_cols].values.astype(np.float32)

X_val = val_df[feature_cols].values.astype(np.float32)
y_val = val_df[target_cols].values.astype(np.float32)

X_test = test_df[feature_cols].values.astype(np.float32)
y_test = test_df[target_cols].values.astype(np.float32)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train).astype(np.float32)
X_val_s   = scaler.transform(X_val).astype(np.float32)
X_test_s  = scaler.transform(X_test).astype(np.float32)

class TabDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return self.X[i], self.y[i]

BATCH_SIZE = 8192
train_loader = DataLoader(TabDataset(X_train_s, y_train), batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(TabDataset(X_val_s, y_val), batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(TabDataset(X_test_s, y_test), batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

### GPU model

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_dim: int, out_dim: int = 3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, out_dim),
        )
    def forward(self, x): return self.net(x)

model = MLP(in_dim=len(feature_cols), out_dim=len(target_cols)).to(DEVICE)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.SmoothL1Loss()

model

### Training loop

In [ ]:
def run_epoch(model, loader, train: bool):
    model.train(train)
    total_loss = 0.0
    n = 0
    for Xb, yb in loader:
        Xb = Xb.to(DEVICE, non_blocking=True)
        yb = yb.to(DEVICE, non_blocking=True)
        if train:
            opt.zero_grad(set_to_none=True)
        pred = model(Xb)
        loss = loss_fn(pred, yb)
        if train:
            loss.backward()
            opt.step()
        bs = Xb.size(0)
        total_loss += loss.item() * bs
        n += bs
    return total_loss / max(n, 1)

EPOCHS = 10
history = []
start = time.time()

for epoch in range(1, EPOCHS+1):
    tr_loss = run_epoch(model, train_loader, train=True)
    va_loss = run_epoch(model, val_loader, train=False)
    history.append({"epoch": epoch, "train_loss": tr_loss, "val_loss": va_loss})
    print(f"Epoch {epoch:02d} | train {tr_loss:.4f} | val {va_loss:.4f}")

elapsed = time.time() - start
print("Elapsed seconds:", elapsed)
print("Device:", DEVICE)

### Eval

In [ ]:
def predict_all(model, loader):
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for Xb, yb in loader:
            Xb = Xb.to(DEVICE, non_blocking=True)
            pred = model(Xb).cpu().numpy()
            preds.append(pred)
            trues.append(yb.numpy())
    return np.vstack(preds), np.vstack(trues)

pred, true = predict_all(model, test_loader)

metrics = {}
for i, name in enumerate(target_cols):
    yhat = pred[:, i]
    y    = true[:, i]
    metrics[name] = {
        "MAE": float(mean_absolute_error(y, yhat)),
        "RMSE": float(mean_squared_error(y, yhat, squared=False)),
    }
metrics

In [ ]:
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("outputs/metrics", exist_ok=True)

torch.save(model.state_dict(), "outputs/models/mlp_state.pt")
pd.DataFrame(history).to_csv("outputs/metrics/history.csv", index=False)
with open("outputs/metrics/test_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

import joblib
joblib.dump(scaler, "outputs/models/feature_scaler.pkl")